# Final Project: Overview

# Objective

The objective of this project is for you to demonstrate your mastery of the Machine Learning process
**using Neural Networks**.



# Submission requirements

The guidelines will be similar to the Midterm
- you will write a procedure that takes raw data and produces predictions

You will submit a *single* model for evaluation.

**Demonstrate that all cells in your notebook work**

The final cell in your notebook should print the message "Done"
- `print("Done")`
- If we run your notebook and this last cell does not execute your submission will be inadequate

## Testing

*You must perform out of sample testing*.

If you want to perform cross-validation in training, that is fine, but you
must *also* test out of sample to show that you are not over-fitting.

It is up to you to create the out of sample data that you feel best evaluates your model.

We will create holdout data (that we will not show you) for grading.

The procedure you write to make predictions should be able to work on the unseen holdout data
(similar to how it should work for your test set but the holdout set has *no targets*)

    

# The data

Data will be provided to you
- as multiple files in a directory which we refer to as a *data directory*

The reason for this is that the different files may convey different information.

You will be responsible for deciding
- which files to use
- which fields within the files to use

We will give you a data directory for training.

# Submission guidelines

Here are the basics, a code template that you must complete is in the following cells
- you will be required to store  your model in a file
- you will be required to write a procedure `MyModel` that takes two arguments
    - `test_dir`
        - this is a *relative path* to the holdout data directory
    - `model_save_file`
        - this is a *relative path* to the file containing your model
- the holdout data directory is similar in structure to the training data directory
    - but without target labels !  It is your job to predict these.
- your procedure must produce predictions given this holdout data directory

This means that your procedure must
- prepare the files in the holdout data directory similar to the way that they were prepared in the training data directory

We will provide you with a sample data directory that will resemble the holdout -- this is so that you
may test the procedure you write for submission.



## Detailed submission guidelines


In **addition to your notebook that trains/evaluates your model**,
- please also submit the file that contains your trained model.
    - use `saveModel` to put your final, trained model in this file
- We will **not** train your model; we will only use the method `MyModel`
    - which **you** will implement
    - and which uses `loadModel` and the name of the file in which the model was saved
    - this will create the model that we will evaluate


Here is a code template for you to complete
- it will save your model (assuming it is in variable `my_model`)
- it provides the specification for procedure `MyModel`, which *you must complete*


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pandas-ta

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_ta as ta
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import joblib

In [ ]:
file_path = '/content/drive/MyDrive/stock_prediction/Data/train/AAPL.csv'
df = pd.read_csv(file_path, index_col='Dt', parse_dates=True)

In [ ]:
df = df.copy()

In [ ]:
os.environ['PYTHONHASHSEED'] = '42'
random.seed(42)
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)


def create_features(df):
    df['Return'] = df['Adj Close'].pct_change()

    df['Momentum_5'] = ta.roc(df['Adj Close'], length=5)

    macd = ta.macd(df['Adj Close'], fast=12, slow=26, signal=9)
    df['MACD'] = macd['MACD_12_26_9']
    df['Signal_Line'] = macd['MACDs_12_26_9']
    df['MACD_Diff'] = df['MACD'] - df['Signal_Line']

    df['RSI'] = ta.rsi(df['Adj Close'], length=14)

    df['SMA_10'] = ta.sma(df['Adj Close'], length=10)
    df['SMA_60'] = ta.sma(df['Adj Close'], length=60)
    df['EMA_10'] = ta.ema(df['Adj Close'], length=10)

    df['Volatility_10'] = df['Adj Close'].rolling(window=10).std()

    bb = ta.bbands(df['Adj Close'], length=20, std=2)
    df['Bollinger_Upper'] = bb['BBU_20_2.0']
    df['Bollinger_Lower'] = bb['BBL_20_2.0']
    df['Bollinger_Mid'] = bb['BBM_20_2.0']
    df['Bollinger_Width'] = df['Bollinger_Upper'] - df['Bollinger_Lower']

    df['ATR'] = ta.atr(high=df['High'], low=df['Low'], close=df['Adj Close'], length=14)

    df['Momentum_10'] = ta.roc(df['Adj Close'], length=10)
    df['SMA_50'] = ta.sma(df['Adj Close'], length=50)
    df['Volatility_Ratio'] = df['Volatility_10'] / df['Adj Close']

    df['Williams_%R'] = ta.willr(high=df['High'], low=df['Low'], close=df['Adj Close'], length=14)

    stoch = ta.stoch(high=df['High'], low=df['Low'], close=df['Adj Close'], k=14, d=3)
    df['Stoch_%K'] = stoch['STOCHk_14_3_3']
    df['Stoch_%D'] = stoch['STOCHd_14_3_3']

    for lag in range(1, 21):
        df[f'Return_Lag_{lag}'] = df['Return'].shift(lag)

    df.dropna(inplace=True)
    return df

def prepare_data(df, feature_columns, train_end_year=2014):
    train_data = df[df.index.year <= train_end_year]
    test_data = df[df.index.year > train_end_year]

    X_train = train_data[feature_columns].copy()
    y_train = train_data['Return'].values.reshape(-1, 1)
    X_test = test_data[feature_columns].copy()
    y_test = test_data['Return'].values.reshape(-1, 1)

    scalers_X = {}
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()

    for col in feature_columns:
        scaler = RobustScaler()
        X_train_scaled[col] = scaler.fit_transform(X_train[[col]].values)
        X_test_scaled[col] = scaler.transform(X_test[[col]].values)
        scalers_X[col] = scaler

    scaler_y = MinMaxScaler(feature_range=(-1, 1))
    y_train_scaled = scaler_y.fit_transform(y_train)
    y_test_scaled = scaler_y.transform(y_test)

    return X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled, scalers_X, scaler_y, train_data, test_data

def create_sequences(data, targets=None, seq_length=90):
    X_seq, y_seq = [], []
    data_values = data.values if isinstance(data, pd.DataFrame) else data
    if targets is not None:
        for i in range(len(data_values) - seq_length):
            X_seq.append(data_values[i:i+seq_length])
            y_seq.append(targets[i+seq_length])
        return np.array(X_seq), np.array(y_seq)
    else:
        for i in range(len(data_values) - seq_length):
            X_seq.append(data_values[i:i+seq_length])
        return np.array(X_seq), None

def build_optimized_lstm_model(input_shape):
    model = Sequential([
        Bidirectional(LSTM(256, return_sequences=True, activation='tanh'), input_shape=input_shape),
        BatchNormalization(),
        Dropout(0.1),
        Bidirectional(LSTM(128, return_sequences=True, activation='tanh')),
        BatchNormalization(),
        Dropout(0.1),
        LSTM(64, return_sequences=False, activation='tanh'),
        BatchNormalization(),
        Dropout(0.1),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.05),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='mse', metrics=['mae'])
    return model

def saveModel(model, model_path):
    model_save_file = model_path + '.keras'
    model.save(model_save_file)
    print(f"Model saved in {model_save_file}; submit with your assignment.")
    return model_save_file


file_path = '/content/drive/MyDrive/stock_prediction/Data/train/AAPL.csv'
df = pd.read_csv(file_path, index_col='Dt', parse_dates=True)
df = create_features(df)

feature_columns = [
    'Return_Lag_1', 'Return_Lag_2', 'Return_Lag_3', 'Return_Lag_4',
    'Return_Lag_5', 'Return_Lag_6', 'Return_Lag_7', 'Return_Lag_8',
    'Return_Lag_9', 'Return_Lag_10', 'Return_Lag_11', 'Return_Lag_12',
    'Return_Lag_13', 'Return_Lag_14', 'Return_Lag_15', 'Return_Lag_16',
    'Return_Lag_17', 'Return_Lag_18', 'Return_Lag_19', 'Return_Lag_20',
    'Momentum_5', 'RSI', 'Signal_Line', 'MACD', 'Bollinger_Width', 'ATR',
    'Stoch_%K', 'Volume', 'Momentum_10', 'Volatility_Ratio'
]

X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled, scalers_X, scaler_y, train_data, test_data = prepare_data(df, feature_columns, train_end_year=2014)

seq_length = 120
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, seq_length=seq_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, seq_length=seq_length)

split_index = int(0.8 * len(X_train_seq))
X_train_fold, X_val_fold = X_train_seq[:split_index], X_train_seq[split_index:]
y_train_fold, y_val_fold = y_train_seq[:split_index], y_train_seq[split_index:]

optimized_model = build_optimized_lstm_model((X_train_seq.shape[1], X_train_seq.shape[2]))
history = optimized_model.fit(
    X_train_fold, y_train_fold,
    validation_data=(X_val_fold, y_val_fold),
    epochs=50,
    batch_size=32,
    verbose=1,
    shuffle=False
)


model_path = "/content/drive/MyDrive/stock_prediction/final_model"
model_save_file = saveModel(optimized_model, model_path)


joblib.dump(scalers_X, "/content/drive/MyDrive/stock_prediction/scalers_X.pkl")
joblib.dump(scaler_y, "/content/drive/MyDrive/stock_prediction/scaler_y.pkl")



def MyModel(test_dir, model_save_file):
    model = load_model(model_save_file)

    scalers_X = joblib.load("/content/drive/MyDrive/stock_prediction/scalers_X.pkl")
    scaler_y = joblib.load("/content/drive/MyDrive/stock_prediction/scaler_y.pkl")

    holdout_file = os.path.join(test_dir, "AAPL.csv")
    df_holdout = pd.read_csv(holdout_file, parse_dates=['Dt'], index_col='Dt')

    df_holdout = create_features(df_holdout)

    X_holdout = df_holdout[feature_columns].copy()

    for col in feature_columns:
        scaler = scalers_X[col]
        X_holdout[col] = scaler.transform(X_holdout[[col]].values)


    X_holdout_seq, _ = create_sequences(X_holdout, targets=None, seq_length=seq_length)

    predictions_scaled = model.predict(X_holdout_seq)
    predictions = scaler_y.inverse_transform(predictions_scaled).flatten()

    ids = df_holdout.index[seq_length:].tolist()
    predictions = predictions.tolist()

    return predictions, ids

print("Done training and setting up MyModel.")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - loss: 1.6695 - mae: 1.0311 - val_loss: 0.0785 - val_mae: 0.2367
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 1.0552 - mae: 0.8058 - val_loss: 0.0812 - val_mae: 0.2301
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.7352 - mae: 0.6845 - val_loss: 0.2608 - val_mae: 0.4213
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.6070 - mae: 0.6128 - val_loss: 0.5025 - val_mae: 0.5965
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 0.5332 - mae: 0.5775 - val_loss: 0.9320 - val_mae: 0.8157
Epoch 6/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.4360 - mae: 0.5285 - val_loss: 0.7235 - val_mae: 0.7049
Epoch 7/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.3659 - mae: 0.4758 - val_loss: 0.6615 - val_mae: 0.6488
Epoch 8/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.3225 - mae: 0.4478 - val_loss: 0.5617 - val_mae: 0.6147
Epoch 9/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 0.282

## Evaluate your model on the holdout data directory

**You must run the following cell** from the directory that contains your model file

Here is how we will evaluate your submission
- we will create a directory whose only content is
    - sub-directory `Data`
- we will restore your model from the saved file`
- we will run the cell in your notebook that should be a copy of the one below
    - it calls procedure `MyModel` with the arguments given below
    - your implementation of `MyModel`
        - must successfully load your model file, *given where **we** have place it as described above*
        - must successfully return one prediction for each example in the holdout directory *given where **we** have placed the holdout directory*

In [ ]:
holdout_dir = os.path.join(".", "Data", "holdout")
predicts = MyModel(holdout_dir, model_save_file)

FileNotFoundError: [Errno 2] No such file or directory: './Data/holdout/AAPL.csv'

In [ ]:
print("Done")